# Language in Space

## Session 10: Spatial autocorrelation and Moran's I

### Gerhard Jäger

January 20, 2022


In [ ]:
options(repr.plot.width=20, repr.plot.height=13)

### Recap homework

sample solution

In [ ]:
library(tidyverse)
library(tmap)
library(sf)

In [ ]:
system("mkdir phoible")

In [ ]:
phoibleZip = "phoible/phoible-v2.0.1.zip"

if(!file.exists(phoibleZip)) {
    download.file(
        "https://zenodo.org/record/2677911/files/cldf-datasets/phoible-v2.0.1.zip",
        dest = phoibleZip
    )
}


In [ ]:
unzip(phoibleZip, exdir="phoible")

In [ ]:
languages <- read_csv("phoible//cldf-datasets-phoible-f36deac/cldf/languages.csv") %>%
    drop_na() %>%
    st_as_sf(coords=c("Longitude", "Latitude"))
st_crs(languages) <- 4326

In [ ]:
languages %>%
    slice_head(n=10)

In [ ]:
parameters <- read_csv("phoible//cldf-datasets-phoible-f36deac/cldf/parameters.csv")
parameters %>%
    slice_head(n=10)

In [ ]:
values <- read_csv("phoible//cldf-datasets-phoible-f36deac/cldf/values.csv")
values %>%
    slice_head(n=10)

In [ ]:
nSegments <- values %>%
    group_by(Language_ID) %>%
    summarise(nSegments = n()) %>%
    arrange(desc(nSegments))
nSegments %>%
    slice_head(n=10)

In [ ]:
options(repr.plot.width=10, repr.plot.height=7)

In [ ]:
nSegments %>% 
    ggplot() +
    geom_histogram(aes(x=nSegments)) +
    scale_x_log10()

In [ ]:
world.160e <- read_sf("data/world_160e.gpkg")

In [ ]:
bg.map <- world.160e %>% 
    st_transform("+proj=eqearth lon_0=160") %>%
    tm_shape() +
    tm_fill()
bg.map

In [ ]:
nSegments.sf <- languages %>%
    transmute(Language_ID = ID) %>%
    inner_join(nSegments)
nSegments.sf %>%
    slice_head(n=10)

In [ ]:
tmap_options(bg.color = "black", legend.text.color = "black")


solution <- bg.map +
    nSegments.sf %>%
    tm_shape() +
    tm_symbols(
        col="nSegments",
        style = "quantile",
        title.col = "Number of segments",
        size=0.1,
        border.lwd=0.02
    ) +
    tm_layout(legend.outside=T, scale=1) 
solution

# Spatial autocorrelation

(material taken from https://mgimond.github.io/Spatial/)


Suppose we have a collection of points on the earth's surface, and each point has a (numerical) feature value. We may ask the question: 

**Is the distribution of feature values random, or is there spatial structure?**

Technically, this amounts to the question whether the **spatial autocorrelation** is 0.


### Temporal autocorrelation

<img src=_img/Acf_new.svg width=500>
(image from Wikipedia)

<img src=_img/Random_maps.png>

**Moran's I** is a test statistic to test for spatial autocorrelation.



In [ ]:
library(spdep)


In [ ]:
load(url("https://github.com/mgimond/Spatial/raw/main/Data/moransI.RData"))


In [ ]:
st_as_sf(s1)


In [ ]:
tm_shape(s1) + 
    tm_polygons(style="quantile", col = "Income") +
    tm_legend(outside = TRUE, text.size = .8) 

First we create a "neighbor list". Two regions are neighbors if they share a boundary. If we set the option `queen=TRUE`, even a shared point is sufficient.

In [ ]:
nbm <- poly2nb(s1, queen=TRUE)
nbm

In [ ]:
plot(s1)
plot.nb(nbm, s1, add=T, col='blue')

The corresponding adjacency matrix:

In [ ]:
nmtx <- nb2mat(nbm)
rownames(nmtx) <- colnames(nmtx) <- s1$NAME
round(nmtx, 2)

Next we assign weights to the edges of the neighborhood graph. For simplicity's sake, we assume equal weight for each neighbor.

In [ ]:
lw <- nb2listw(nbm, style="W", zero.policy=TRUE)

In [ ]:
lw$weights[1]

Then we compute the weighted average of the incomes of neighboring counties for each county.

In [ ]:
Inc.lag <- lag.listw(lw, s1$Income)

In [ ]:
st_as_sf(s1) %>%
    st_drop_geometry() %>%
    select(NAME, Income) %>%
    mutate(Inc.lag = Inc.lag)

Doing some exploratory data analysis:

In [ ]:
st_as_sf(s1) %>%
    st_drop_geometry() %>%
    select(NAME, Income) %>%
    mutate(Inc.lag = Inc.lag) %>%
    ggplot() +
    geom_point(aes(x=Income, y=Inc.lag)) +
    geom_smooth(aes(x=Income, y=Inc.lag), method=lm)

The slope of the regression line is Moran's I.



Spelling out the mathematics:

Let $N$ be the number of points and $w_{ij}$ the *weight* (strength of influence) between points $i$ and $j$. $w_{ii} =0$ for all $i$. 

$$
\begin{aligned}
I &= \frac{N}{W} \frac{\sum_{i,j} w_{ij}(x_i-\overline{x})(x_j-\overline{x})}{\sum_i(x_i-\overline{x})^2}\\
W &= \sum_{ij} w_{ij}\\
\overline x &= \frac{\sum_i x_i}{N}
\end{aligned}
$$

$I$ ranges from $-1$ to $1$. The expected value in the absence of spatial autocorrelation is 
$
\frac{-1}{N-1}
$

In [ ]:
M <- lm(Inc.lag ~ s1$Income)
summary(M)

In [ ]:
coef(M)[2]

To assess significance, we can the a random permutation test.

In [ ]:
n <- 599L   # Define the number of simulations
I.r <- vector(length=n)  # Create an empty vector

for (i in 1:n){
  # Randomly shuffle income values
  x <- sample(s1$Income, replace=FALSE)
  # Compute new set of lagged values
  x.lag <- lag.listw(lw, x)
  # Compute the regression slope and store its value
  M.r    <- lm(x.lag ~ x)
  I.r[i] <- coef(M.r)[2]
}

In [ ]:
data.frame(I.r = I.r) %>%
    ggplot() +
    geom_histogram(aes(x=I.r)) +
    geom_vline(xintercept=coef(M)[2], col='red')


Pseudo-$p$-value:

In [ ]:
mean(I.r > coef(M)[2])

The Moran test does not use permutations but computes the $p$-value analytically.

In [ ]:
moran.test(s1$Income,lw)

There is also a version of Moran's I test using simulations:


In [ ]:
MC<- moran.mc(s1$Income, lw, nsim=599)

# View results (including p-value)
MC

In [ ]:
# Plot the distribution (note that this is a density plot instead of a histogram)
plot(MC, main="", las=1)

## Application to German dialect data



In [ ]:
pad_voronoi <- read_sf("data/pad_voronoi.shp")

In [ ]:
pad_voronoi %>%
    ggplot() +
    geom_sf(fill=pad_voronoi$col)

In [ ]:
pad_voronoi %>%
    ggplot() +
    geom_sf(aes(fill=r)) +
    scale_fill_distiller(palette = "Spectral")

In [ ]:
pad_voronoi %>%
    ggplot() +
    geom_sf(aes(fill=g)) +
    scale_fill_distiller(palette = "Spectral")
    


In [ ]:
pad_voronoi %>%
    ggplot() +
    geom_sf(aes(fill=b)) +
scale_fill_distiller(palette = "Spectral")

Just from eyeballing, it seems likely that the three MDS-dimensions have a strong spatial autocorrelation. Let us test this using Moran's I.

In [ ]:
pad_nb <- poly2nb(pad_voronoi, queen=TRUE)

In [ ]:
pad_nb

In [ ]:
pad_voronoi %>%
    st_geometry() %>%
    st_union() %>%
    plot()
plot(pad_nb, as(pad_voronoi, 'Spatial'), add=T)

Side remark: The neighbor relation obtained from the Voronoi tesselation of a set of points is the [Delaunay triangulation](https://en.wikipedia.org/wiki/Delaunay_triangulation).

In [ ]:
pad_lw <- nb2listw(pad_nb, style="W", zero.policy = T)

### First dimension ("r")

In [ ]:
moran.test(pad_voronoi$r, pad_lw)

In [ ]:
(pad_mc <- moran.mc(pad_voronoi$r, pad_lw, nsim=1000))

In [ ]:
plot(pad_mc)

### Second dimension ("g")

In [ ]:
moran.test(pad_voronoi$g, pad_lw)

In [ ]:
(pad_mc <- moran.mc(pad_voronoi$g, pad_lw, nsim=1000))

In [ ]:
plot(pad_mc)

### Third dimension ("b")

In [ ]:
moran.test(pad_voronoi$b, pad_lw)

In [ ]:
(pad_mc <- moran.mc(pad_voronoi$b, pad_lw, nsim=1000))

In [ ]:
plot(pad_mc)

### a look at the linear regression

In [ ]:
r.lag = lag.listw(pad_lw, pad_voronoi$r)
g.lag = lag.listw(pad_lw, pad_voronoi$g)
b.lag = lag.listw(pad_lw, pad_voronoi$b)

In [ ]:
st_as_sf(pad_voronoi) %>%
    st_drop_geometry() %>%
    select(r) %>%
    mutate(r.lag = r.lag) %>%
    ggplot() +
    geom_point(aes(x=r, y=r.lag)) +
    geom_smooth(aes(x=r, y=r.lag), method=lm)

In [ ]:
st_as_sf(pad_voronoi) %>%
    st_drop_geometry() %>%
    select(g) %>%
    mutate(g.lag = g.lag) %>%
    ggplot() +
    geom_point(aes(x=g, y=g.lag)) +
    geom_smooth(aes(x=g, y=g.lag), method=lm)

In [ ]:
st_as_sf(pad_voronoi) %>%
    st_drop_geometry() %>%
    select(b) %>%
    mutate(b.lag = b.lag) %>%
    ggplot() +
    geom_point(aes(x=b, y=b.lag)) +
    geom_smooth(aes(x=b, y=b.lag), method=lm)

## Definitions of *neighborhood*

The value of Moran's I (and many other things in geostatistics) depend on how *neighborhood* and *neighborhood weights* are defined. In the following we will explore various common options and their impact on Moran's I.

Let us forget the Voronoi polygons for the time being and return to the original point data.

In [ ]:
pad = read_csv("data/pad_mds.csv")

pad %>% slice_head(n=10)

In [ ]:
coo <- pad %>%
    select(LONGITUDE, LATITUDE) %>%
    as.matrix()

In [ ]:
S.dist  <-  dnearneigh(coo, 0, 80, longlat=T) 

In [ ]:
S.dist

In [ ]:
pad_sf <- st_as_sf(pad, coords=c("LONGITUDE", "LATITUDE"))

In [ ]:
pad_voronoi %>%
    st_geometry() %>%
    st_union() %>%
    plot()
plot.nb(S.dist, st_geometry(pad_sf), add=T)

In [ ]:
lw <- nb2listw(S.dist, style="W",zero.policy=T) 

In [ ]:
MI  <-  moran.mc(pad$r, lw, nsim=999,zero.policy=T) 

In [ ]:
plot(MI, main="", las=1) 

In [ ]:
MI

## Spatial correlograms

The choice of the distance threshold for neighborhood seems arbitrary. 

A **correlogram** computes Moran's I for different distance bins and plots them. This allows to assess the radius of effective spatial autocorrelation.

First step: define a function that computes Moran's I for a given distance interval.

In [ ]:
dist2mi <- function(x, coo, lower, upper) {
    S.dist  <-  dnearneigh(coo, lower, upper, longlat=T) 
    lw <- nb2listw(S.dist, style="W",zero.policy=T) 
    return(moran.mc(x, lw, nsim=999,zero.policy=T))
}

In [ ]:
dist2mi(pad$r, coo, 0, 10)

Second step: computer Moran's I for each bin.

In [ ]:
nBins <- 80
binWidth <- 10
binCenters <- binWidth*(1:nBins)-binWidth/2

In [ ]:
crlg.r <- c()
for (i in 1:nBins) {
    binCenter <- binCenters[i]
    MI <- dist2mi(pad$r, coo, binCenter-5, binCenter+5)
    mi <- as.numeric(MI$statistic)
    signif <- MI$p.value < 0.05
    crlg.r <- rbind(crlg.r, c(binCenter, mi, signif))
}

Third step: convert the results into a tibble and plot them with `ggplot`. Significance of the Moran test is indicated by color.

In [ ]:
df.r <- tibble(data.frame(crlg.r))
colnames(df.r) <- c("distance", "I", "significant")
df.r$significant <- as.character(df.r$significant)


In [ ]:
library(repr)
options(repr.plot.width=30, repr.plot.height=10)

In [ ]:
 df.r %>%
    ggplot() +
    geom_point(aes(x=distance, y=I, col=significant),size=8) +
    geom_hline(yintercept = 0) +
    theme(text=element_text(size=32,  family="Comic Sans MS")) +
    geom_smooth(aes(x=distance, y=I), method="gam", se=F)


Same procedure for the `b` and `g` variables:

In [ ]:
crlg.g <- c()
for (i in 1:nBins) {
    binCenter <- binCenters[i]
    MI <- dist2mi(pad$g, coo, binCenter-5, binCenter+5)
    mi <- as.numeric(MI$statistic)
    signif <- MI$p.value < 0.05
    crlg.g <- rbind(crlg.g, c(binCenter, mi, signif))
}
df.g <- tibble(data.frame(crlg.g))
colnames(df.g) <- c("distance", "I", "significant")
df.g$significant <- as.character(df.g$significant)
df.g %>%
    ggplot() +
    geom_point(aes(x=distance, y=I, col=significant),size=8) +
    geom_hline(yintercept = 0) +
    theme(text=element_text(size=32,  family="Comic Sans MS")) +
    geom_smooth(aes(x=distance, y=I), method="gam", se=F)


In [ ]:
crlg.b <- c()
for (i in 1:nBins) {
    binCenter <- binCenters[i]
    MI <- dist2mi(pad$b, coo, binCenter-5, binCenter+5)
    mi <- as.numeric(MI$statistic)
    signif <- MI$p.value < 0.05
    crlg.b <- rbind(crlg.b, c(binCenter, mi, signif))
}
df.b <- tibble(data.frame(crlg.b))
colnames(df.b) <- c("distance", "I", "significant")
df.b$significant <- as.character(df.b$significant)
df.b %>%
    ggplot() +
    geom_point(aes(x=distance, y=I, col=significant),size=8) +
    geom_hline(yintercept = 0) +
    theme(text=element_text(size=32,  family="Comic Sans MS")) +
    geom_smooth(aes(x=distance, y=I), method="gam", se=F)



## Local Moran's I

For the next topic, I will use the Delauney triangulation (i.e., the neihborhood relation derived from the Voronoi tesslation).

Let us re-consider the linear regression that let to Moran's I. We can quantify for each observation how strongly it contributes to the regression coefficient.


In [ ]:
pad_nb <- poly2nb(pad_voronoi, queen=TRUE)
pad_lw <- nb2listw(pad_nb, style="W", zero.policy = T)

In [ ]:
options(repr.plot.width=10, repr.plot.height=7)

In [ ]:
pad_voronoi %>% 
    ggplot() +
    geom_point(aes(x=r, y=r.lag), size=5, col='white') +
    geom_smooth(aes(x=r, y=r.lag), method='lm', se=F, col='blue') +
    theme_dark()
    



The formula for Moran's I is

$$
\begin{aligned}
I &= \frac{N}{W} \frac{\sum_{i,j} w_{ij}(x_i-\overline{x})(x_j-\overline{x})}{\sum_i(x_i-\overline{x})^2}\\
& = \frac{N}{W\sum_j(x_j-\overline{x})^2} \sum_i (x_i-\overline{x})\sum_j w_{ij}(x_j-\overline{x})
\end{aligned}
$$

Leaving out the first $\sum$ gives us the individual contributions of each data point.

$$
\begin{aligned}
I_i &\propto (x_i-\overline{x})\sum_j w_{ij}(x_j-\overline{x})
\end{aligned}
$$

In [ ]:
pad_voronoi %>% 
    mutate(li.r = localmoran(pad_voronoi$r, pad_lw)[,1]) %>%
    ggplot() +
    geom_point(aes(x=r, y=r.lag, col=li.r), size=5) +
    scale_color_gradient2(
    low = "grey", 
    mid = "white", 
    high = "brown", 
    midpoint = 1
    ) + 
    geom_hline(yintercept = mean(pad_voronoi$r), col='lightgrey')+
    geom_vline(xintercept = mean(pad_voronoi$r), col='lightgrey')+
    geom_smooth(aes(x=r, y=r.lag), method='lm', se=F, col='blue') +
    theme_dark()
    



*Local Moran's I* is a measure of the degree to which the value at a point can be predicted from its neighborhood.

In [ ]:
pad_voronoi %>% 
    mutate(li.r = localmoran(pad_voronoi$r, pad_lw)[,1]) %>%
    ggplot() +
    geom_sf(aes(fill=li.r)) +
    scale_fill_gradient2(
    low = "grey", 
    mid = "white", 
    high = "brown", 
    midpoint = 1
  ) +theme_dark()
    



The same procedure for the other two dimensions.

In [ ]:
pad_voronoi %>% 
    mutate(li.g = localmoran(pad_voronoi$g, pad_lw)[,1]) %>%
    ggplot() +
    geom_sf(aes(fill=li.g)) +
    scale_fill_gradient2(
    low = "grey", 
    mid = "white", 
    high = "brown", 
    midpoint = 1
  ) + theme_dark()
    



In [ ]:
pad_voronoi %>% 
    mutate(li.b = localmoran(pad_voronoi$b, pad_lw)[,1]) %>%
    ggplot() +
    geom_sf(aes(fill=li.b)) +
    scale_fill_gradient2(
    low = "grey", 
    mid = "white", 
    high = "brown", 
    midpoint = 2
  ) + theme_dark()
    